In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from ydata_profiling import ProfileReport
from src.data import make_dataset
import glob

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [31]:
df_raw = pd.concat(
    (
        pd.read_excel(archivo, usecols=make_dataset.COLS_A_OCUPAR.keys())
        for archivo in glob.glob("../data/raw/*.xlsx")
    )
)

df_raw.loc[:, make_dataset.COLS_A_HASHEAR] = df_raw.loc[:, make_dataset.COLS_A_HASHEAR].apply(
    make_dataset.hashear_columna_texto
)


In [66]:
df_raw["Código Diagnóstico"] = df_raw["Código Diagnóstico"].astype(str)


In [9]:
df_limpia = pd.read_csv("../data/interim/datos_limpios.csv", encoding="latin-1", sep=";")


# Analisis exploratorio previo al preprocesamiento

In [77]:
report = ProfileReport(df_raw)
report.to_file("../reports/1.0-jrb-exploratory-data-analysis.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Analisis de Columna Codigo Diagnostico

In [78]:
largo_codigos = df_raw["Código Diagnóstico"].str.len().value_counts(normalize=True, dropna=False)
display(largo_codigos * 100)


5    58.598147
3    27.038000
7     9.194450
2     1.983357
6     1.550353
4     1.053433
1     0.581900
8     0.000360
Name: Código Diagnóstico, dtype: float64

- El 58.6% de los codigos presenta largo 5. Los codigos con largo 5 son del tipo: "C38.4" o "C34_1", por
lo que es el formato deseable

In [79]:
df_raw[df_raw["Código Diagnóstico"].str.len() == 5]["Código Diagnóstico"].sample(10)


114354    Q20.8
31929     J84.8
52439     I49.8
20199     D19.0
25754     G47.3
135700    C34_9
105071    J44.9
258498    I49.8
179645    I06.0
77610     E84.0
Name: Código Diagnóstico, dtype: object


- El 27.0% de los codigos presenta largo 3. Los codigos con largo 3 son del tipo: "R05", "C34", por
lo que corresponde a la Categoria del diagnostico.


In [80]:
largo_3 = df_raw[df_raw["Código Diagnóstico"].str.len() == 3][
    ["Código Diagnóstico", "Nombre Diagnóstico"]
].value_counts()
display(largo_3)

Código Diagnóstico  Nombre Diagnóstico                                               
I99                 Otros trastornos y los no especificados del sistema circulatorio     29603
J47                 Bronquiectasia                                                       15305
I50                 Insuficiencia cardíaca                                               13381
C34                 Tumor maligno de los bronquios y del pulmón                          11886
J84                 Otras enfermedades pulmonares intersticiales                         11185
                                                                                         ...  
I13                 Hipertensión cardíaca y enfermedad renal.                                1
R96                 Otras muertes súbitas de causa desconocida                               1
H60                 Otitis externa                                                           1
H55                 Nistagmo y otros movimientos oculares i

- El 9.19% de los codigos presenta largo 7. Los codigos con largo 7 son del tipo: "J44.8.4" o "E10-E16".

In [81]:
largo_7 = df_raw[df_raw["Código Diagnóstico"].str.len() == 7]["Código Diagnóstico"].sample(10)
display(largo_7)


160895    J45.0.1
89552     J80-J84
141761    J45.0.3
202608    J45.8.1
85569     J45.1.1
196294    J45.1.1
59375     J45.0.1
105250    J45.1.1
39615     J43.9.1
156380    J45.0.1
Name: Código Diagnóstico, dtype: object

- El 1.98% de los codigos presenta largo 2. Los codigos con largo 2 son del tipo: "55", por lo que
tienen una notacion de diagnostico deficiente. Sin embargo, estos codigos se relacionan a un
unico "Nombre Diagnostico". Esto puede permitir recodificar estos codigos de forma mas facil.

In [82]:
largo_2 = df_raw[df_raw["Código Diagnóstico"].str.len() == 2][
    ["Código Diagnóstico", "Nombre Diagnóstico"]
].value_counts()
display(largo_2)

Código Diagnóstico  Nombre Diagnóstico                    
55                  Fibrosis pulmonar                         4335
79                  Trasplante  Pulmon                        2788
78                  Trasplante Cardiaco                       1197
41                  Nódulo pulmonar                            842
20                  Bronquiectasias                            363
54                  Enfermedad pulmonar difusa                 342
42                  Tumor pulmonar                             113
77                  Asma Bronquial en el Adulto (Sospecha)     110
74                  Tabaquismo                                 100
64                  Hipertensión pulmonar primaria              99
17                  Neumonitis por hipersensibilidad            98
49                  Disnea en estudio                           79
44                  Cáncer pulmonar                             53
21                  Bronquiectasias infectadas                  50
59 

- El 1.55% de los codigos presenta largo 6. Los codigos con largo 6 son del tipo: "300-23". Tambien tienen
una relacion 1:1 entre Código Diagnóstico y Nombre Diagnóstico.

In [83]:
largo_6 = df_raw[df_raw["Código Diagnóstico"].str.len() == 6][
    ["Código Diagnóstico", "Nombre Diagnóstico"]
].value_counts()
display(largo_6)

Código Diagnóstico  Nombre Diagnóstico                                                             
300-23              Tetralogía de Fallot (TOF)                                                         2523
300-32              Ventriculo Unico Funcional                                                         1385
300-28              Transposición Grandes Arterias (discordancia de la conexión ventriculoarterial)     848
300-29              Transposición Grandes Arterias + CIV                                                519
300-12              Doble Salida VD (DSVD)                                                              517
                                                                                                       ... 
902-12              Preexcitación ventricular                                                             2
I51.14              CIV Sub Aórtica (tipo fallot)                                                         2
902-27              Macroreentrada p

- El 1.05% de los codigos presenta largo 4 y son del tipo: "81-4" o "I32*"

In [84]:
largo_4 = df_raw[df_raw["Código Diagnóstico"].str.len() == 4][
    ["Código Diagnóstico", "Nombre Diagnóstico"]
].value_counts()
display(largo_4)

Código Diagnóstico  Nombre Diagnóstico                                     
81-4                Otro                                                       4553
88-1                Otro:                                                       300
81-6                Estenosis Pulmonar Valvular                                 254
91-3                Enfermedad Coronaria 3V                                     251
85-6                Cardiomiopatía Isquemica                                    126
91-1                Enfermedad Coronaria 1V                                      80
85-3                Enfermedad Coronaria (cardiopatía coronaria)                 73
91-2                Enfermedad Coronaria 2V                                      66
91-5                Aneurisma Aorta Ascendnte (sin mención de ruptura)           43
91-6                Aneurisma Raíz Aórtica (sin mención de ruptura)              25
81-5                Estenosis Pulmonar Subvalvular (Infundibular)                16


- El 0.58% de los codigos presenta largo 1, y son del tipo "5" o "6"

In [85]:
largo_1 = df_raw[df_raw["Código Diagnóstico"].str.len() == 1][
    ["Código Diagnóstico", "Nombre Diagnóstico"]
].value_counts()
display(largo_1)

Código Diagnóstico  Nombre Diagnóstico          
5                   Otro                            3072
6                   Ex TBC pulmonar                  107
2                   Crisis asmática                   23
7                   TBC pulmonar                      23
8                   TBC pulmonar multiresistente       5
9                   Micobacteriosis atipica            2
dtype: int64

- El 0.00036% de los codigos presenta largo 8, y son del tipo "Z00- Z13"

In [86]:
largo_8 = df_raw[df_raw["Código Diagnóstico"].str.len() == 8][
    ["Código Diagnóstico", "Nombre Diagnóstico"]
].value_counts()
display(largo_8)

Código Diagnóstico  Nombre Diagnóstico                                  
Z00- Z13            PRUEBAS PARA ACLARAR O INVESTIGAR PROBLEMAS DE SALUD    2
dtype: int64

## Preprocesamiento de Diagnostico

- Teniendo en cuenta los resultados previos, una propuesta tentativa para el preprocesamiento
de los codigos de diagnosticos es:

    1. Eliminar el simbolo "."
    2. Eliminar los espacios
    2. A los que tienen largo 3, agregar una X al final hasta tener un largo 4
    3. A los que tienen largo 7, se separaran en diags con largo 5 (ej: J4591) y largo 7 (ej: E65-E68).
    Los que sigan teniendo largo 7 corresponderan a los Capitulos, por lo que sera necesario observar
    que hacer con esos diagnosticos (ya que son muy poco especificos)
    4. A los que tienen largo 2, 6, 4 y 1 se puede crear un diccionario para transformar el codigo a un
    codigo cie

# Analisis de base preprocesada

## Analisis de Columna Codigo Diagnostico

In [113]:
largos = df_limpia["Código Diagnóstico"].str.len().value_counts().index


In [120]:
resultados = {}
for largo in largos:
    filtro_por_largo = df_limpia["Código Diagnóstico"].str.len() == largo

    resumen_diags_con_largo = df_limpia[filtro_por_largo][
        ["Código Diagnóstico", "Nombre Diagnóstico"]
    ].value_counts()

    resultados[f"largo_{largo}"] = resumen_diags_con_largo

    print(f"Los codigos con largo {largo} tienen la siguiente forma:")
    print(resumen_diags_con_largo.sample(30, replace=True).index.get_level_values(0))
    print()

Los codigos con largo 4 tienen la siguiente forma:
Index(['O994', 'M795', 'K219', 'I258', 'J960', 'M130', 'J041', 'Z228', 'J181',
       'C548', 'I350', 'N188', 'C341', 'I132', 'J069', 'Q393', 'D150', 'J631',
       'C418', 'E441', 'G724', 'I793', 'Y409', 'J370', 'C151', 'O010', 'Q340',
       'L941', 'N398', 'G724'],
      dtype='object', name='Código Diagnóstico')

Los codigos con largo 3 tienen la siguiente forma:
Index(['R55', 'N18', 'N04', 'C78', 'W34', 'P27', 'Z03', 'K70', 'D83', 'I74',
       'I45', 'R09', 'I46', 'S29', 'J20', 'M07', 'H60', 'E05', 'Z34', 'Q67',
       'A46', 'M32', 'K42', 'G96', 'Q33', 'L58', 'I22', 'Q32', 'G47', 'J60'],
      dtype='object', name='Código Diagnóstico')

Los codigos con largo 5 tienen la siguiente forma:
Index(['200-9', '200-1', 'D4811', '300-9', 'I5115', '91-26', 'J4582', '300-8',
       'D4811', 'J4501', 'J4393', '300-7', 'J4311', 'J4591', '200-6', '300-7',
       '300-4', '903-3', 'I5110', '904-5', '905-2', '902-3', '903-1', '900-5',
       'D

In [121]:
with pd.ExcelWriter("../data/interim/diagnosticos_encontrados.xlsx") as file:
    for nombre_archivo, df_a_guardar in resultados.items():
        df_a_guardar.to_excel(file, sheet_name=nombre_archivo)